### Testing audio tokenization

In [2]:
from transformers import AutoProcessor
from transformers import EncodecModel
import torch
import torch.nn.functional as F
import torchaudio

from data_processing.data_pipeline import AudioExample

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
model = EncodecModel.from_pretrained("facebook/encodec_32khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_32khz")


def convert_audio_to_codes(audio_values: torch.Tensor):
    inputs = processor(
        raw_audio=audio_values.squeeze(),
        sampling_rate=processor.sampling_rate,
        return_tensors="pt",
    )
    output = model.encode(
        inputs["input_values"].to(device), inputs["padding_mask"].to(device)
    )
    return output.audio_codes.squeeze()


def convert_codes_to_audio(audio_codes: torch.Tensor):
    audio_codes = audio_codes.unsqueeze(0).unsqueeze(0)
    waveform = model.decode(audio_codes, [None])
    return waveform.audio_values.detach()[0]


# waveform, sample_rate = torchaudio.load(
#     "audio_dataset/shift2repeat2/sax_sk9rbWPDLxw.wav"
# )
# token_tensor = convert_wav_to_tensor(waveform)
# recreated_wav = convert_tensor_to_wav(token_tensor)
# torchaudio.save("recreated.wav", recreated_wav, sample_rate)


c:\Users\bucks\anaconda3\envs\encodec\Lib\site-packages\transformers\models\encodec\modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [19]:
out = torch.load("audio_dataset/augmented_audio_small.pt", map_location="cpu", weights_only=False)
eg: AudioExample = out[0]
eg.lead.shape


torch.Size([1, 9304064])

In [10]:
torchaudio.save("bking_recreated.wav", convert_codes_to_audio(backing[0]), 32000)
torchaudio.save("lead_recreated.wav", convert_codes_to_audio(lead[0]), 32000)

In [15]:
examples = torch.load("audio_dataset/augmented_audio_small.pt", weights_only=False)
torchaudio.save("bking.wav", examples[1].backing, 32000)
torchaudio.save("lead.wav", examples[1].lead, 32000)

In [3]:
backing_full = []
lead_full = []
for indices in [(0, 400), (400, 1000), (1000, "")]:
    start, end = indices
    backing, lead = torch.load(f"audio_dataset/codes/augmented_codes_{start}_{end}.pt")
    backing_full.extend(backing)
    lead_full.extend(lead)

In [8]:
len(lead_full), len(backing_full)

(1710, 1710)

In [7]:
torch.save((backing_full, lead_full), "audio_dataset/codes/aug_codes.pt")